In [ ]:
!pip install langchain openai tiktoken rapidocr-onnxruntime langchain-community langchain-openai faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.9/14.9 MB 79.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 70.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.8/84.8 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 66.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 44.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 80.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 978.2/978.2 kB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 3.1 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the so

In [ ]:
from google.colab import userdata
OPENAI_API_KEY  = userdata.get("OPENAI_API_TOKEN")

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

DATA INGESTION

In [ ]:
import langchain

In [ ]:
from langchain_community.document_loaders import TextLoader, CSVLoader
from langchain_community.vectorstores import FAISS

In [ ]:
template_loader = CSVLoader("Own funds template.csv")

In [9]:
rulebook = TextLoader("Own Funds.txt")
instructions_loader = TextLoader("Own Funds Instructions.txt")

docs = rulebook.load() + instructions_loader.load() + template_loader.load()

In [10]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [11]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 600, chunk_overlap = 60)

In [12]:
chunks = text_splitter.split_documents(docs)

In [13]:
from langchain_openai import OpenAIEmbeddings, ChatOpenAI

In [14]:
embeddings = OpenAIEmbeddings(openai_api_key = OPENAI_API_KEY)
vectorstore = FAISS.from_documents(chunks, embeddings)
retriever = vectorstore.as_retriever()

In [15]:
template = """
You are a PRA COREP Reporting Assistant. Your goal is to map a bank's financial scenario
into the correct COREP template cells based on the PRA Rulebook and Instructions.

RETRIEVED KNOWLEDGE (Rules, Instructions, and Template Structure):
{context}

REPORTING SCENARIO:
{scenario}

USER QUESTION:
{question}

You must respond ONLY in the following JSON format:
{{
    "template_mapping": [
        {{
            "template_code": "e.g. C 01.00",
            "row": "e.g. 0130",
            "column": "e.g. 0010",
            "item_name": "Name of the field",
            "value_to_report": "The number from the scenario",
            "rule_reference": "Specific paragraph from Rulebook"
        }}
    ],
    "validation_rules": "Briefly list any basic consistency checks (e.g. Row X must be sum of Y and Z).",
    "missing_data_flag": "List any information missing from the scenario needed to be 100% sure.",
    "audit_log": "A professional justification explaining WHY these cells were chosen, citing the specific PRA Rulebook and Instruction sections."
}}
"""

In [16]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import JsonOutputParser

In [17]:
prompt = ChatPromptTemplate.from_template(template)
llm = ChatOpenAI(model="gpt-4-turbo", temperature=0)
chain = prompt | llm | JsonOutputParser()


In [28]:
my_scenario = """
Bank Beta has 'Paid up capital instruments' of £10,000,000.
However, the bank also has £1,500,000 in 'Goodwill' recorded on its balance sheet
from the purchase of a smaller firm last year.
"""
my_question = "How should the Goodwill be reported in the C 01.00 template, and does it affect our CET1 capital?"

retrieved_docs = retriever.invoke(my_question)

result = chain.invoke({
    "context": retrieved_docs,
    "scenario": my_scenario,
    "question": my_question
})

# 4. Print the result
print(result["template_mapping"])
print(result["audit_log"])

[{'template_code': 'C 01.00', 'row': '0320', 'column': '0010', 'item_name': '(-) Goodwill included in the valuation of significant investments', 'value_to_report': '1500000', 'rule_reference': 'Point (b) of Article 37 and Article 43 CRR'}]
The value of Goodwill (£1,500,000) is reported in row 0320 of the C 01.00 template as per the instructions found in the Own Funds Instructions.txt, specifically under the section detailing the reporting requirements for Goodwill as per Point (b) of Article 37 and Article 43 CRR. This row is specifically designated for Goodwill included in the valuation of significant investments, aligning with the scenario where Bank Beta acquired a smaller firm. The reported value matches the balance sheet figure, ensuring compliance with the regulatory requirements.


In [29]:
print(result)

{'template_mapping': [{'template_code': 'C 01.00', 'row': '0320', 'column': '0010', 'item_name': '(-) Goodwill included in the valuation of significant investments', 'value_to_report': '1500000', 'rule_reference': 'Point (b) of Article 37 and Article 43 CRR'}], 'validation_rules': 'Ensure that the reported value of Goodwill in row 0320 matches the amount recorded on the balance sheet. Additionally, check that the total deductions from CET1 capital are correctly summed and reported in the relevant total deductions row.', 'missing_data_flag': 'None', 'audit_log': 'The value of Goodwill (£1,500,000) is reported in row 0320 of the C 01.00 template as per the instructions found in the Own Funds Instructions.txt, specifically under the section detailing the reporting requirements for Goodwill as per Point (b) of Article 37 and Article 43 CRR. This row is specifically designated for Goodwill included in the valuation of significant investments, aligning with the scenario where Bank Beta acqui

In [23]:
import pandas as pd

In [30]:
print("========================================")
print("     COREP REPORTING TEMPLATE EXTRACT   ")
print("========================================\n")

mapping_data = result.get("template_mapping", [])

if mapping_data:
    df = pd.DataFrame(mapping_data)
    df.columns = ["Template", "Row", "Col", "Description", "Value", "Rule Ref"]
    print(df.to_string(index=False))
else:
    print("No mapping found for this scenario.")

#Validation and Missing Data
print("\n" + "-"*40)
print("VALIDATION & AUDIT TRAIL")
print("-"*40)

print(f"VALIDATION CHECK: {result.get('validation_rules', 'N/A')}")
print(f"MISSING DATA:     {result.get('missing_data_flag', 'None')}")

print("\n--- JUSTIFICATION (AUDIT LOG) ---")
print(result.get("audit_log", "No justification provided."))
print("-"*40)

     COREP REPORTING TEMPLATE EXTRACT   

Template  Row  Col                                                       Description   Value                                   Rule Ref
 C 01.00 0320 0010 (-) Goodwill included in the valuation of significant investments 1500000 Point (b) of Article 37 and Article 43 CRR

----------------------------------------
VALIDATION & AUDIT TRAIL
----------------------------------------
VALIDATION CHECK: Ensure that the reported value of Goodwill in row 0320 matches the amount recorded on the balance sheet. Additionally, check that the total deductions from CET1 capital are correctly summed and reported in the relevant total deductions row.
MISSING DATA:     None

--- JUSTIFICATION (AUDIT LOG) ---
The value of Goodwill (£1,500,000) is reported in row 0320 of the C 01.00 template as per the instructions found in the Own Funds Instructions.txt, specifically under the section detailing the reporting requirements for Goodwill as per Point (b) of Article 37 a